In [2]:
#!pip install transformers faiss-cpu torch

In [6]:
#####################################3

In [11]:
import torch
import faiss
import numpy as np
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRContextEncoderTokenizer, DPRQuestionEncoderTokenizer

# Load the DPR models and tokenizers
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Load the corresponding tokenizers
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# Function to encode query
def encode_query(query):
    inputs = question_tokenizer(query, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        query_embeddings = question_encoder(**inputs).pooler_output
    return query_embeddings

# Function to encode documents (texts)
def encode_documents(documents):
    context_embeddings = []
    for doc in documents:
        inputs = context_tokenizer(doc, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            context_embedding = context_encoder(**inputs).pooler_output
        context_embeddings.append(context_embedding)
    return context_embeddings

# Load documents from a text file (assuming each line is a separate document)
with open('Employee_info.txt', 'r') as file:
    documents = file.readlines()

# Encode documents
document_embeddings = encode_documents(documents)

# Example query
query = "least year of experience"

# Encode the query
query_embedding = encode_query(query)

# Convert torch tensors to numpy arrays
query_vector = query_embedding.numpy().flatten()
document_vectors = [doc.numpy().flatten() for doc in document_embeddings]

# Create a FAISS index for fast retrieval
index = faiss.IndexFlatL2(len(query_vector))  # L2 distance metric for cosine similarity

# Add document vectors to the index
index.add(np.array(document_vectors))

# Search the index for the closest match (k=1 to get the best match)
D, I = index.search(np.array([query_vector]), k=1)  # k=1 to get the closest document

# Retrieve the most relevant document
best_document = documents[I[0][0]]
print(f"Most relevant document: {best_document}")


Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the

Most relevant document: Q: Which employee has the least years of experience and what is their designation?

